# An introduction to the `sequence-jacobian` toolkit

This notebook serves as an introduction to the `sequence-jacobian` toolkit and the classes and main functions it provides for solving dynamic general equilibrium models in sequence space.

This introduction will cover the following topics:
0. `sequence-jacobian` preliminaries
    - The `sequence-jacobian` toolkit directory structure
    - Importing `sequence-jacobian`
1. How do `Block` objects work?
    1. `SimpleBlock`
    2. `HetBlock`
    3. `SolvedBlock`
2. The main functions of `sequence-jacobian`
    - `steady_state`
    - `get_G`
    - `td_map`

The notebook accompanies the working paper by Auclert, Bardóczy, Rognlie, Straub (2019): "Using the Sequence-Space Jacobian to Solve and Estimate Heterogeneous-Agent Models". Please see the [Github repository](https://github.com/shade-econ/sequence-jacobian) for more information and code.

Also, be sure to check out the other model notebooks in this directory to see some applications of `sequence-jacobian`.

# `sequence-jacobian` preliminaries

##  0.1 The `sequence-jacobian' directory structure 

We will only illustrate a smaller subset of the directory structure for the sake of clarity.

```
sequence-jacobian/
│
├──notebooks/
│  ├─ intro_to_sequence_jacobian.ipynb
│  └─ rbc.ipynb
│  
└──sequence_jacobian/
   ├─ blocks/
   │    ├─ simple
   │    ├─ het
   │    ├─ helper
   │    └─ solved
   │
   ├─ models/
   │    ├─ rbc
   │    ├─ krusell_smith
   │    └─ hank
   │
   ├─ utilities/
   │    ├─ discretize
   │    ├─ graph
   │    └─ devtools
   │
   ├─ jacobian
   ├─ nonlinear
   └─ steady_state
```

The `sequence-jacobian` repository contains a few directories at the top-level.

The notable ones for the purposes of this introduction are:
- `notebooks`: where example notebooks demonstrate the `sequence-jacobian` functionality on well-known models
- `sequence_jacobian`: the main python package containing the source code.

Within `sequence_jacobian`, we have:
- Modules implementing the main steps of the solution method: `jacobian`, `nonlinear`, `steady_state`
- Sub-directories containing:
    - `blocks`: definitions of the various `Block` objects, which wrap model equations
    - `models`: implemented models as collections of `Block` objects
    - `utilities`: useful functions that support user in debugging their code and that improve workflow within the `sequence-jacobian` source code

##  0.2 Importing the `sequence-jacobian` code

There are a few different routes to importing the `sequence_jacobian` package properly. Depending on your desired environment or what you find convenient, make sure you follow one of these procedures first, or else the `sequence_jacobian` package will not load properly for you:

1. **(Recommended) Using `pip`**: You can install `sequence-jacobian` using `pip`, python's native package-management system, by executing the following command in your python console with `pip` loaded: `pip install git+https://github.com/shade-econ/sequence-jacobian@develop-cai`. Alternatively you can append `python -m ` to the front of that command and execute it directly in the terminal.
2. **Using sys.path.append()**: You can directly clone the `sequence-jacobian` repository from Github [here](https://github.com/shade-econ/sequence-jacobian/tree/develop-cai), import the `sys` package inside your python console, and call `sys.path.append(<the/path/to/sequence_jacobian>)`. E.g. if you want to run this notebook, you can call `sys.path.append(..)`.
3. **Using `conda`**: While `conda` does not have a straightforward way of building from source like `pip`, if you prefer to work within `conda` environments, you can create a `conda` environment and then use `pip` as above to install `sequence-jacobian` within your `conda` environment.

In [17]:
import numpy as np

import sequence_jacobian as sj
from sequence_jacobian import simple, het, solved
from sequence_jacobian import utilities as utils

# 1 How do `Block` objects work?

## 1.A `SimpleBlock`

The simplest python class that `sequence-jacobian` provides is called the `SimpleBlock`.

These blocks are intended for use in defining the model equations that are solely functions of parameters and economy-wide *aggregate* variables as in the following equilibrium conditions from the representative firm problem in the real business cycle model:

$$ 
\begin{align}
r_t + \delta &= \alpha Z_t \left(\frac{K_{t-1}}{L_t} \right)^{\alpha-1} \\
w_t &= (1-\alpha) Z_t \left(\frac{K_{t-1}}{L_t} \right)^{\alpha} \\
Y_t &= Z_t K_{t-1}^\alpha L_t^{1 - \alpha}
\end{align}
$$

`SimpleBlock`, like the other main `Block` classes, has a few **attributes** that describe its contents:
- `inputs`
- `outputs`
- `.f`

A custom **constructor**, in the form of a python decorator, `@simple`.

And three main **methods**:
- `.ss`, which is used in the steady state computation
- `.td`, which is used in the transitional dynamics
- `.jac`, which is used in the Jacobian computation

In [2]:
@simple
def my_first_simple_block(a, b, c):
    d = a + b
    e = b * c
    return d, e

The attributes describe what variables `my_first_simple_block` takes as inputs, what variables it outputs, and provide a way to invoke the function it is defined over.

In [3]:
my_first_simple_block.inputs

{'a', 'b', 'c'}

In [4]:
my_first_simple_block.outputs

{'d', 'e'}

In [5]:
my_first_simple_block.f

<function __main__.my_first_simple_block(a, b, c)>

Hence, we can directly invoke the function that `my_first_simple_block` is defined over as if it were a simple python function

In [6]:
my_first_simple_block.f(1, 2, 3)

(3, 6)

Also as seen above, the custom constructor requires no additional arguments and can simply be used by annotating the function on the line above with the decorator `@simple` as is standard in python.

Because dynamic general equilibrium models are, by definition, *dynamic*, typically the equations that define such models contain time displacements.

We can represent the equations $d_t = a_{t+1} + b_t$ and $e_t = b_{t+1} * c_{t-1}$ as the following `SimpleBlock`, using Dynare-like syntax

In [7]:
@simple
def my_first_dynamic_block(a, b, c):
    d = a(1) + b
    e = b(1) * c(-1)
    return d, e

Notice here, we cannot simply invoke `my_first_dynamic_block` as if it were a simple python function because `a(1)` is not standard python syntax. Under the hood, `sequence-jacobian` catches these inputs and wraps them in special classes to handle time displacement prior to invoking the function to capture information relevant for the steady state, jacobian, etc. which we will get to.

In [13]:
try:
    my_first_dynamic_block.f(1, 2, 3)
except TypeError:
    print("Catching the error - `TypeError: 'int' object is not callable`."
          "\nGenerally, don't call the `.f` method directly!")

Catching the error - `TypeError: 'int' object is not callable`.
Generally, don't call the `.f` method directly!


The error occurs because the syntax `a(1)` is not valid python syntax, when `a` is a number. In general, we would not advise the user to directly invoke the `.f` attribute but rather work with the main methods of the `SimpleBlock`: `.ss`, `.td`, `.jac`.

As mentioned above, the main way the user should interface with `SimpleBlock`s directly (which itself will typically be unnecessary once we get to the main functions in the `sequence-jacobian` toolkit) is through the methods `.ss`, `.td`, and `.jac`.

These methods utilize the information encoded in the `SimpleBlock` both by the algebraic relationships written in the variable definitions but also in the time displacements, as in `my_first_dynamic_block` above.

For example, in `.ss` the dynamic equation $d_t = a_{t+1} + b_t$ would become $d^* = a^* + b^*$, where the $*$ variables indicate their steady state values, whereas in `.td` or `.jac`, the information from the dynamic equation is retained so that transitions or Jacobians are calculated with the correct time displacements.

The main functions in the `sequence-jacobian` toolkit provide the main handling of these time displacements, so we will not elaborate on this further here.

Below are some other examples of more complicated equations contained in `SimpleBlock`s that demonstrate their flexibility.

We are also accommodate the use of more arbitrary functions called on the variables that enter into `SimpleBlock`s, with `.apply()` for single argument functions and `apply_function()` for functions with multiple arguments.

**Note**: As of now `apply_function()` has more limited functionality, only working within the context of transitional dynamics and *not* Jacobian calculation. Either reference the source code for more information, or otherwise proceed with caution!

In [9]:
from sequence_jacobian import apply_function

@simple
def heres_a_block(a, b, c):
    d = (a(-1) ** b)/c
    e = -d(+1)
    return d, e

@simple
def heres_another_one(a, b, c):
    d = a(-1).apply(np.log)
    e = b(+1) + c(+1)
    return d, e

def foo(a, b, c):
    return np.sin(a) + b ** c

@simple
def and_another(a, b, c):
    d = apply_function(foo, a, b, c)
    return d

# And finally a more realistic one
@simple
def firm(K, L, Z, alpha, delta):
    r = alpha * Z * (K(-1) / L) ** (alpha-1) - delta
    w = (1 - alpha) * Z * (K(-1) / L) ** alpha
    Y = Z * K(-1) ** alpha * L ** (1 - alpha)
    return r, w, Y

## 1.B `HetBlock`

The next class that `sequence-jacobian` provides is the `HetBlock`. While it is nice to be able to work with aggregate variables, one of the main selling points of the `sequence-jacobian` toolkit is that it provides a fast, general framework for solving heterogeneous agent models. 

Hence, the `HetBlock` provides a class for building heterogeneous-agent components that are compatible with `SimpleBlock`s to incrementally build up a dynamic general equilibrium model.

`HetBlock` shares the same set of primary attributes and methods as `SimpleBlock` but is constructed in a different manner with a few extra ingredients. Let's inspect the following `HetBlock` taken from the `krusell_smith.ipynb` notebook.

In [18]:
def household_init(a_grid, e_grid, r, w, eis):
    coh = (1 + r) * a_grid[np.newaxis, :] + w * e_grid[:, np.newaxis]
    Va = (1 + r) * (0.1 * coh) ** (-1 / eis)
    return Va


@het(exogenous='Pi', policy='a', backward='Va', backward_init=household_init)
def household(Va_p, Pi_p, a_grid, e_grid, r, w, beta, eis):
    """Single backward iteration step using endogenous gridpoint method for households with CRRA utility.
    Parameters
    ----------
    Va_p     : array (S*A), marginal value of assets tomorrow
    Pi_p     : array (S*S), Markov matrix for skills tomorrow
    a_grid   : array (A), asset grid
    e_grid   : array (A), skill grid
    r        : scalar, ex-post real interest rate
    w        : scalar, wage
    beta     : scalar, discount rate today
    eis      : scalar, elasticity of intertemporal substitution
    Returns
    ----------
    Va : array (S*A), marginal value of assets today
    a  : array (S*A), asset policy today
    c  : array (S*A), consumption policy today
    """
    uc_nextgrid = (beta * Pi_p) @ Va_p
    c_nextgrid = uc_nextgrid ** (-eis)
    coh = (1 + r) * a_grid[np.newaxis, :] + w * e_grid[:, np.newaxis]
    a = utils.interpolate.interpolate_y(c_nextgrid + a_grid, coh, a_grid)
    utils.optimized_routines.setmin(a, a_grid[0])
    c = coh - a
    Va = (1 + r) * c ** (-1 / eis)
    return Va, a, c

The `HetBlock` constructor `@het` wraps the "backward iteration" equation, i.e. equation (10) from the paper:

$$ \mathbf{v}_t = \mathcal{v}(\mathbf{v}_{t+1}, \mathbf{X}_t) $$

This equation defines the mapping between the value function, $\mathbf{v}_{t+1}$, and aggregate input variables, $\mathbf{X}_t$, to the value function $\mathbf{v}_t$ in the previous period. While there are numerous ways to write this mapping, in the above code we employ the endogenous gridpoint method.

Now to describe each of the keyword arguments for `@het`:
- **exogenous**: The Markov transition matrix for the evolution of the exogenous, individual-level states. In the above case $e_t$, the skill level of households.
- **policy**: The endogenous, individual-level state variable[s]. In the above case $a_t$, the asset holdings of households.
- **backward**: The "value function" variable on which to perform the backward iteration. In the above case $\frac{\partial}{\partial a} V_{t+1}(a, e)$, the marginal value of assets tomorrow.
- **backward_init**: A function mapping variables/parameters to an initial value for the "value function" variable to commence backward iteration.

While  `@het` decorates the backward iteration function, which seems to solely define a mapping between *individual-level* variables, i.e. mapping $\frac{\partial}{\partial a}V_{t+1}(a, e)$ to $\frac{\partial}{\partial a}V_t(a, e)$ and calculating $c_t(a, e)$ and $a^\prime_t(a, e)$ as byproducts, the `HetBlock` itself should still be thought of as a mapping between *aggregate* variables. 

**Why?** Recall we have a set of aggregate inputs $\mathbf{X}_t$, which enter into the backward iteration equation $\mathbf{v}_t = \mathcal{v}(\mathbf{v}_{t+1}, \mathbf{X}_t)$. After performing the full set of backward iteration evaluations, we then perform a set of forward iterations to evaluate how the distribution of endogenous and exogenous states evolves over time, given by equation (11) from the paper:

$$ \mathbf{D}_{t+1} = \Lambda(\mathbf{v}_{t+1}, \mathbf{X}_t)^\prime \mathbf{D}_t $$

Finally, we use those distribution values $\mathbf{D}_t$ to aggregate up the individual-level variables, i.e. with $\mathcal{y}_t(\mathbf{v}_{t+1}, \mathbf{X}_t) := \{c_t(a, e)$, $a^\prime_t(a, e)\}$ we have equation (12) from the paper:

$$ \mathbf{Y}_t = \mathcal{y}_t(\mathbf{v}_{t+1}, \mathbf{X}_t)^\prime D_t $$

Hence because all of this computation is encapsulated within evaluations of the `HetBlock`, we end up with a proper aggregate-to-aggregate mapping $\mathbf{X}_t \rightarrow \mathbf{Y}_t$.

**For further reading on working with heterogeneous-agent model components**:
- See Section 3: Computing Jacobians for heterogeneous-agent problems in the paper for further details on the algorithm embedded inside `HetBlock` evaluations.
- See the `het_jacobian.ipynb` notebook for a more in-depth view of the internal computations happening within `HetBlock` evaluations.

## 1.C `SolvedBlock`

Sometimes within the structure of the DAG we can specify a set of equations that constitute a smaller, self-contained DAG. One example of this is the New Keynesian Phillips Curve, which is a `SolvedBlock` in our two-asset heterogeneous agent model.

In [20]:
@solved(unknowns={'pi': (-0.1, 0.1)}, targets=['nkpc'], solver="brentq")
def pricing_solved(pi, mc, r, Y, kappap, mup):
    nkpc = kappap * (mc - 1/mup) + Y(+1) / Y * (1 + pi(+1)).apply(np.log) / (1 + r(+1)) - (1 + pi).apply(np.log)
    return nkpc

@simple
def pricing(pi, mc, r, Y, kappap, mup):
    nkpc = kappap * (mc - 1/mup) + Y(+1) / Y * (1 + pi(+1)).apply(np.log) / (1 + r(+1)) - (1 + pi).apply(np.log)
    return nkpc

Here we can solve for inflation, `pi`, to ensure that the target equation, `nkpc`, is satisfied without having to reference equations in other `Block` objects. This is in contrast to specifying the same equation as a `SimpleBlock`, which would require the user to include the unknown `pi` and target `nkpc` at the top level of the DAG with the rest of the unknowns and targets.

The advantage of this approach is that it improves computational efficiency by streamlining the DAG evaluation process. For example, instead of asking the root-finding algorithm to solve an $n$-dimensional problem in a DAG without a `SolvedBlock`, the root-finding algorithm would instead only have to solve a reduced problem, where along the evaluation path it would additionally solve the unidimensional root-finding problem provided by the `SolvedBlock`. While it's not absolutely necessary to break out every possible `SolvedBlock` within the DAG, it may improve performance for large-scale models.

# 2 The main functions of `sequence-jacobian`

## 2.A `steady_state`